In [1]:
import os
import requests
import json

In [2]:
JIRA_URL = os.environ.get('JIRA_URL')
JIRA_USER = os.environ.get('JIRA_USER')
JIRA_KEY = os.environ.get('JIRA_KEY')

In [3]:
def extract_data(jqlQuery, fields):
  start = 0
  max = 100
  issues = []

  while True:
    requestBody = {
        'jql': jqlQuery,
        'fields': fields,
        'startAt': start,
        'maxResults': max
    }

    response = requests.post(f'{JIRA_URL}/rest/api/3/search', json=requestBody, auth=(JIRA_USER, JIRA_KEY))
    responseJSON = json.loads(response.text)

    issues.extend(responseJSON['issues'])
    start += max

    if start > responseJSON['total']:
      return issues


In [4]:
ASSESSMENT_JQL_QUERY = 'project = DA AND issuetype = Assessment ORDER BY created DESC'
ASSESSMENT_FIELDS = [
    'status',
    'customfield_13408',  # Digital Service Key
    'customfield_13390',  # Phase
    'customfield_13389',  # Type
    'customfield_13401',  # Result
    'customfield_13421',  # Requires reassessment
    'customfield_13411',  # Booking date
    'customfield_13416',  # Closing date
    'customfield_13407',  # Assessment date
    'customfield_13414',  # Material due date
    'customfield_13412',  # Pre-chat date
    'customfield_13163',  # DSS:Accessible
    'customfield_13164',  # DSS:Agile
    'customfield_13159',  # DSS:Consistent
    'customfield_13272',  # DSS:Data
    'customfield_13160',  # DSS:Design
    'customfield_13167',  # DSS:Encourage
    'customfield_13161',  # DSS:First Time
    'customfield_13169',  # DSS:Measurre
    'customfield_13170',  # DSS:Ministerr
    'customfield_13165',  # DSS:Open
    'customfield_13166',  # DSS:Privacy
    'customfield_13168',  # DSS:Support
    'customfield_13158',  # DSS:Team
    'customfield_13162',  # DSS:Test
    'customfield_13157'  # DSS:Understand
]

In [5]:
raw_assessments = extract_data(ASSESSMENT_JQL_QUERY, ASSESSMENT_FIELDS)

In [6]:
import pandas as pd

In [7]:
def transform_assessment_data(json_data):
  # Parse JSON into dataframe
  assessments = pd.json_normalize(json_data)

  # Log number records at start
  print(
      f'Dataframe loaded with {assessments.shape[0]} rows and {assessments.shape[1]} columns')

  keep_columns = [
      'key',
      'fields.status.name',
      'fields.customfield_13408',  # Digital Service Key
      'fields.customfield_13390.value',  # Phase
      'fields.customfield_13389.value',  # Type
      'fields.customfield_13401.value',  # Result
      'fields.customfield_13421.value',  # Requires reassessment
      'fields.customfield_13411',  # Booking date
      'fields.customfield_13416',  # Closing date
      'fields.customfield_13407',  # Assessment date
      'fields.customfield_13414',  # Material due date
      'fields.customfield_13412',  # Pre-chat date
      'fields.customfield_13163.value',  # DSS:Accessible
      'fields.customfield_13164.value',  # DSS:Agile
      'fields.customfield_13159.value',  # DSS:Consistent
      'fields.customfield_13272.value',  # DSS:Data
      'fields.customfield_13160.value',  # DSS:Design
      'fields.customfield_13167.value',  # DSS:Encourage
      'fields.customfield_13161.value',  # DSS:First Time
      'fields.customfield_13169.value',  # DSS:Measurre
      'fields.customfield_13170.value',  # DSS:Ministerr
      'fields.customfield_13165.value',  # DSS:Open
      'fields.customfield_13166.value',  # DSS:Privacy
      'fields.customfield_13168.value',  # DSS:Support
      'fields.customfield_13158.value',  # DSS:Team
      'fields.customfield_13162.value',  # DSS:Test
      'fields.customfield_13157.value'  # DSS:Understand
  ]

  assessment_field_map = {
      'key': 'assessment_key',
      'fields.status.name': 'status',
      'fields.customfield_13408': 'digital_service_key',
      'fields.customfield_13390.value': 'phase',
      'fields.customfield_13389.value': 'type',
      'fields.customfield_13401.value': 'result',
      'fields.customfield_13421.value': 'reassessment',
      'fields.customfield_13411': 'booking_date',
      'fields.customfield_13416': 'closing_date',
      'fields.customfield_13407': 'assessment_date',
      'fields.customfield_13414': 'materials_due_date',
      'fields.customfield_13412': 'pre_chat_date',
      'fields.customfield_13163.value': 'dss_accessible',
      'fields.customfield_13164.value': 'dss_agile',
      'fields.customfield_13159.value': 'dss_consistent',
      'fields.customfield_13272.value': 'dss_data',
      'fields.customfield_13160.value': 'dss_design',
      'fields.customfield_13167.value': 'dss_encourage',
      'fields.customfield_13161.value': 'dss_ensure',
      'fields.customfield_13169.value': 'dss_measure',
      'fields.customfield_13170.value': 'dss_minister',
      'fields.customfield_13165.value': 'dss_open',
      'fields.customfield_13166.value': 'dss_privacy',
      'fields.customfield_13168.value': 'dss_support',
      'fields.customfield_13158.value': 'dss_team',
      'fields.customfield_13162.value': 'dss_test',
      'fields.customfield_13157.value': 'dss_understand'
  }

  column_order = [
      'assessment_key',
      'digital_service_key',
      'phase',
      'type',
      'status',
      'result',
      'reassessment',
      'booking_date',
      'closing_date',
      'assessment_date',
      'materials_due_date',
      'pre_chat_date',
      'dss_accessible',
      'dss_agile',
      'dss_consistent',
      'dss_data',
      'dss_design',
      'dss_encourage',
      'dss_ensure',
      'dss_measure',
      'dss_minister',
      'dss_open',
      'dss_privacy',
      'dss_support',
      'dss_team',
      'dss_test',
      'dss_understand'
  ]

  # Keep only the columns that we need
  assessments = assessments[keep_columns]

  # Rename columns uusing assessment_fields_map
  assessments.rename(columns=assessment_field_map, errors='raise', inplace=True)

  # Reorder columns
  assessments = assessments[column_order]

  # Set datetime columns
  assessments['booking_date'] = pd.to_datetime(assessments['booking_date'])
  assessments['closing_date'] = pd.to_datetime(assessments['closing_date'])
  assessments['assessment_date'] = pd.to_datetime(assessments['assessment_date'])
  assessments['materials_due_date'] = pd.to_datetime(assessments['materials_due_date'])
  assessments['pre_chat_date'] = pd.to_datetime(assessments['pre_chat_date'])

  # Handle reassessment results
  def set_reassessment_bool(str):
    if str == 'Yes':
      return True
    else:
      return False

  assessments['reassessment'] = assessments['reassessment'].apply(
      set_reassessment_bool)

  # Handle incompletes
  incomplete_assessments = assessments['status'] != 'Complete'
  assessments['result'] = assessments['result'].mask(incomplete_assessments, 'Not complete')

  assessments['dss_minister'] = assessments['dss_minister'].fillna('Not applicable')
  assessments['dss_encourage'] = assessments['dss_encourage'].fillna('Not applicable')
  assessments.loc[:, 'dss_accessible':'dss_understand'] = assessments.loc[:, 'dss_accessible':'dss_understand'].fillna('Not complete')

  return assessments


In [8]:
cleaned_assessments = transform_assessment_data(raw_assessments)

Dataframe loaded with 460 rows and 96 columns


In [9]:
cleaned_assessments.head(10)

,assessment_key,digital_service_key,phase,type,status,result,reassessment,booking_date,closing_date,assessment_date,...,dss_encourage,dss_ensure,dss_measure,dss_minister,dss_open,dss_privacy,dss_support,dss_team,dss_test,dss_understand
0,DA-1202,DA-1200,After-discovery,Advice only,Pre-assessment,Not complete,False,2023-03-08,NaT,NaT,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
1,DA-1199,DA-717,Mid-beta,Standard,Pre-assessment,Not complete,False,2023-03-07,NaT,2023-06-13,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
2,DA-1198,DA-1151,After-discovery,Standard,Pre-assessment,Not complete,False,2023-03-02,NaT,2023-04-12,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
3,DA-1197,DA-945,Mid-beta,Standard,Pre-assessment,Not complete,False,2023-02-28,NaT,2023-06-07,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
4,DA-1196,DA-502,Before-live,Standard,Cancelled,Not complete,False,2023-02-22,2023-03-02,2023-05-10,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
5,DA-1194,DA-1136,After-discovery,Standard,Pre-assessment,Not complete,False,2023-02-15,NaT,2023-06-14,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
6,DA-1190,DA-1189,Mid-beta,Standard,Pre-assessment,Not complete,False,2023-02-09,NaT,2023-04-18,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
7,DA-1188,DA-1171,After-discovery,Standard,In Progress,Not complete,False,2023-02-08,NaT,2023-02-22,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
8,DA-1187,DA-1126,Pre-discovery,Advice only,Complete,Advice only,False,2023-02-07,2023-02-24,2023-02-24,...,Not applicable,Not applicable,Course correct to meet the standard,Not applicable,Not applicable,Not applicable,Not applicable,Meets standard,Not applicable,Meets standard
9,DA-1186,DA-1007,After-alpha,Standard,Pre-assessment,Not complete,False,2023-02-03,NaT,2023-06-27,...,Not applicable,Not complete,Not complete,Not applicable,Not complete,Not complete,Not complete,Not complete,Not complete,Not complete
